In [4]:
from eventregistry import EventRegistry, TopicPage
from dotenv import load_dotenv
import os

load_dotenv()
EVENT_REGISTRY_API_KEY = os.getenv("EVENT_REGISTRY_API_KEY")
er = EventRegistry(apiKey = EVENT_REGISTRY_API_KEY)

In [5]:
topic_page = TopicPage(er)
topic_page.loadTopicPageFromER("be2534a5-e15e-42d8-b939-dcba8269a85c")

In [ ]:

# retrieve the first page of 100 results. To get the next page, call with page=2, ...
res = topic_page.getArticles(page = 1)
for art in res.get("articles", {}).get("results", []):
    print(art)